This notebook explores the use of attention for text classification, comparing a model that represents a document by averaging its word embeddings to one that uses an attention mechanism to compute a weighted average over those embeddings.

In [ ]:
import keras
import numpy as np
from sklearn import preprocessing
from keras.layers import Dense, Input, Embedding, Lambda, Layer, Multiply, Dropout, Dot
from keras.models import Model
from keras import backend as K
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
import pandas as pd

In [ ]:
def load_embeddings(filename, max_vocab_size):

    vocab={}
    embeddings=[]
    with open(filename) as file:
        
        cols=file.readline().split(" ")
        num_words=int(cols[0])
        size=int(cols[1])
        embeddings.append(np.zeros(size))  # 0 = 0 padding if needed
        embeddings.append(np.zeros(size))  # 1 = UNK
        vocab["_0_"]=0
        vocab["_UNK_"]=1
        
        for idx,line in enumerate(file):

            if idx+2 >= max_vocab_size:
                break

            cols=line.rstrip().split(" ")
            val=np.array(cols[1:])
            word=cols[0]
            
            embeddings.append(val)
            vocab[word]=idx+2

    return np.array(embeddings), vocab, size

In [ ]:
def read_data(filename, vocab):
    X=[]
    Y=[]
    with open(filename, encoding="utf-8") as file:
        for line in file:
            cols=line.rstrip().split("\t")
            label=cols[0]
            # assumes text is already tokenized
            text=cols[1].split(" ")
            X.append(text)
            Y.append(label)
    return X, Y

In [ ]:
def get_word_ids(docs, vocab, max_length=200):
    
    doc_ids=[]
    
    for doc in docs:
        wids=[]
        for token in doc[:max_length]:
            val = vocab[token.lower()] if token.lower() in vocab else 1
            wids.append(val)
        
        # pad each document to constant width
        for i in range(len(wids),max_length):
            wids.append(0)

        doc_ids.append(wids)

    return np.array(doc_ids)

If you haven't downloaded the glove vectors, do so first -- the top 50K words in the "Common Crawl (42B)"  vectors (300-dimensional) can be found here: [glove.42B.300d.50K.txt](https://drive.google.com/file/d/1n1jt0UIdI3CD26cY1EIeks39XH5S8O8M/view?usp=sharing); download it and place  in your `data` directory.

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file="../data/glove.42B.300d.50K.txt"
glove_in_w2v_format="../data/glove.42B.300d.50K.w2v.txt"
_ = glove2word2vec(glove_file, glove_in_w2v_format)

In [ ]:
embeddings, vocab, embedding_size=load_embeddings("../data/glove.42B.300d.50K.w2v.txt", 50000)

In [ ]:
# Change this to the directory with your data (from the CheckData_TODO.ipynb exercise).  
# The directory should contain train.tsv, dev.tsv and test.tsv
directory="../data/lmrd"

In [ ]:
trainText, trainY=read_data("%s/train.tsv" % directory, vocab)
devText, devY=read_data("%s/dev.tsv" % directory, vocab)

In [ ]:
trainX = get_word_ids(trainText, vocab, max_length=200)
devX = get_word_ids(devText, vocab, max_length=200)

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(trainY)
Y_train=np.array(le.transform(trainY))
Y_dev=np.array(le.transform(devY))

First, let's try a simple model that represents a document by averaging the embeddings for the words it contains.  We'll again use appropriate masking to accommodate zero-padded sequences.

In [ ]:
class MaskedAveragePooling1D(Layer):
    def __init__(self, **kwargs):
        self.supports_masking = True
        super(MaskedAveragePooling1D, self).__init__(**kwargs)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            mask = K.repeat(mask, x.shape[-1])
            mask = tf.transpose(mask, [0,2,1])
            # zero out the elements of x that are masked
            x = x * mask
            
        # sum the modified input, but normalize only over the number of non-masked time steps
        return K.sum(x, axis=1) / K.sum(mask, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[2])

In [ ]:
def get_embedding_average(embeddings):

    vocab_size, word_embedding_dim=embeddings.shape
    
    word_sequence_input = Input(shape=(None,), dtype='int32')
    
    word_embedding_layer = Embedding(vocab_size,
                                    word_embedding_dim,
                                    weights=[embeddings],
                                    mask_zero=True,
                                    trainable=False)

    
    embedded_sequences = word_embedding_layer(word_sequence_input)
    x=MaskedAveragePooling1D()(embedded_sequences)
    
    predictions=Dense(1, activation="sigmoid")(x)

    model = Model(inputs=word_sequence_input, outputs=predictions)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    
    return model

In [ ]:
embedding_model=get_embedding_average(embeddings)
print (embedding_model.summary())

In [ ]:
model=embedding_model

modelName="embedding_model.hdf5"
checkpoint = ModelCheckpoint(modelName, monitor='val_loss', verbose=0, save_best_only=True, mode='min')

model.fit(trainX, Y_train, 
            validation_data=(devX, Y_dev),
            epochs=30, batch_size=128,
            callbacks=[checkpoint])

Next, let's add attention to that simple model to learn a *weighted* average over words---giving more weight to words in the document that are more important for representing the document for the purpose of this classification.

In [ ]:
class AttentionLayerMasking(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(AttentionLayerMasking, self).__init__(**kwargs)


    def build(self, input_shape):
        input_embedding_dim=input_shape[-1]
        
        self.kernel = self.add_weight(name='kernel', 
                            shape=(input_embedding_dim,1),
                            initializer='uniform',
                            trainable=True)
        super(AttentionLayerMasking, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        
        # dot product 
        x=K.dot(x, self.kernel)
        # exponentiate
        x=K.exp(x)
        
        # zero out elements that are masked
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            mask = K.expand_dims(mask, axis=-1)
            x = x * mask
        
        # normalize by sum
        x /= K.sum(x, axis=1, keepdims=True)
        x=K.squeeze(x, axis=2)

        return x

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1])

In [ ]:
def get_embedding_with_attention_masking(embeddings):

    vocab_size, word_embedding_dim=embeddings.shape
    
    word_sequence_input = Input(shape=(None,), dtype='int32')
    
    word_embedding_layer = Embedding(vocab_size,
                                    word_embedding_dim,
                                    weights=[embeddings], 
                                    mask_zero=True,
                                    trainable=False)

    
    embedded_sequences = word_embedding_layer(word_sequence_input)
    
    # first let's transform each word embedding into a new vector to use for measuring its importance
    attention_key_dim=300
    attention_input=Dense(attention_key_dim, activation='tanh')(embedded_sequences)

    # next we'll pass those transformed inputs through an attention layer, getting back a normalized
    # attention value a_i for each token i; \forall i, 0 <= a_i <= 1; for a document with N words, 
    # \sum_{i=0}^N a_i = 1
    
    attention_output = AttentionLayerMasking(word_embedding_dim, name="attention")(attention_input)
    
    # now let's multiply those attention weights by original inputs to get a weighted average over them
    document_representation = Lambda(lambda x: K.batch_dot(x[0], x[1], axes=1), name='dot')([attention_output,embedded_sequences])

    x=Dense(1, activation="sigmoid")(document_representation)

    model = Model(inputs=word_sequence_input, outputs=x)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
    
    return model

In [ ]:
embedding_attention_model=get_embedding_with_attention_masking(embeddings)
print (embedding_attention_model.summary())

In [ ]:
model=embedding_attention_model

modelName="embedding_attention_model.hdf5"
checkpoint = ModelCheckpoint(modelName, monitor='val_loss', verbose=0, save_best_only=True, mode='min')

model.fit(trainX, Y_train, 
            validation_data=(devX, Y_dev),
            epochs=30, batch_size=128,
            callbacks=[checkpoint])

Now let's explore what words in a document a learned attention model is attending to.  

In [ ]:
# load the best saved model

model=embedding_attention_model
model.load_weights("embedding_attention_model.hdf5")

In [ ]:
def analyze(model, doc):
    
    words=doc.split(" ")
    text = get_word_ids([words], vocab, max_length=len(words))
   
    inp = model.input                                    
    outputs = [layer.output for layer in model.layers[1:]]       
    functor = K.function([inp, K.learning_phase()], outputs) 

    test = text[0]
    orig=words
    attention_weights=[]
    test=test.reshape((1,len(words)))
    layer_outs = functor([test, 0.])

    # in this model, attention is the third layer
    attention_layer=layer_outs[2]
    
    for i in range(len(words)):
        val=attention_layer[0,i]
        attention_weights.append(val)
        print ("%.3f\t%s" % (val, orig[i]))
        
    df = pd.DataFrame({'words':orig, 'attention':attention_weights})
    ax = df.plot.bar(x='words', y='attention', figsize=(10,4))

In [ ]:
text="i love this movie !"
analyze(model, text)

In [ ]:
text="i do not love this movie !"
analyze(model, text)